In [1]:
from imutils.paths import list_files
import sqlite3 as sql
from tqdm import tqdm 
import json
import time
import os

In [2]:
ROOT_PATH = f'/home/jovyan/project/DoveNest/informations'
DB_PATH   = f'{ROOT_PATH}/db'
JSON_PATH = f'{ROOT_PATH}/jsons'

In [3]:
load_json = lambda path: json.loads(open(path, 'r').read())

In [4]:
class _DB:
    
    def __init__(self, db_name, table_name, columns):
        
        self.db_name    = db_name
        self.columns    = columns
        self.table_name = table_name
        
        self.conn, self.cursor = self.conn_db()
        
        
    def conn_db(self):
        
        conn   = sql.connect(f'{DB_PATH}/{self.db_name}.db')
        cursor = conn.cursor()
        
        return conn, cursor
    
    
    def create_table(self):
        
        query = f'''
                    CREATE TABLE IF NOT EXISTS {self.table_name}(
                    
                '''
        
        for idx, column in enumerate(self.columns, 1):
            
            name, dtype, is_null = column
            q      = f'{name} {dtype}' if is_null == False \
                    else f'{name} {dtype} NOT NULL'
            query += f'\t{q},\n' if idx != len(self.columns) \
                     else f'{q})'

        self.cursor.execute(query)
        self.conn.commit()
    
    
    def insert_table(self, data_tuple):
        
        q = ''
        for idx, data in enumerate(data_tuple, 1):
            q += '?, ' if idx != len(data_tuple) else '?'
            
        query = f'INSERT INTO {self.table_name} VALUES({q})'
        
        self.cursor.execute(query, data_tuple)
        self.conn.commit()
        
    
    def backup_table(self):
        
        with self.conn:
            with open(f'{DB_PATH}/{self.db_name}.sql', 'w') as f:
                
                for line in self.conn.iterdump(): f.write('%s\n' % line)
                print(f'[INFO] 테이블 <{self.table_name}> 백업이 완료되었습니다. ')

In [5]:
json_paths = sorted(list_files(JSON_PATH))
test_json  = load_json(json_paths[0])

In [6]:
DB = _DB('test', 'games', [['appid', 'INTEGER', True], 
                           ['name',  'TEXT', True], 
                           ['json_data', 'json', True]])

In [7]:
DB.create_table()

In [8]:
appid, name = test_json['steam_appid'], test_json['name']
DB.insert_table([appid, name, json.dumps(test_json)])

In [9]:
s = time.time()
data = load_json(json_paths[0])
time.time() - s

0.001753091812133789

In [10]:
s = time.time()
q = DB.cursor.execute(f'SELECT * FROM games')
data = q.fetchone()
time.time() - s

0.0008060932159423828

In [11]:
data[0]

10

In [12]:
DB.backup_table()

[INFO] 테이블 <games> 백업이 완료되었습니다. 


In [13]:
len(json_paths)

137078

In [14]:
s = time.time()

for json_path in tqdm(json_paths):
    
    try:
        json_data   = load_json(json_path)
        appid, name = json_data['steam_appid'], json_data['name']
        DB.insert_table([appid, name, json.dumps(json_data)])
        
    except Exception as e: print(f'{e}\n')

print(f'[INFO] DB 저장 완료 ({time.time() - s:,.3f}s)')

  5% 6735/137078 [1:30:37<29:13:50,  1.24it/s] 


KeyboardInterrupt: 

In [1]:
a = ['idx'] + ['appid', 'name', 'json_data']
', '.join(a)

'idx, appid, name, json_data'

In [2]:
', '.join('%s' for _ in enumerate(a))

'%s, %s, %s, %s'